In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Install required packages and system dependencies
!wget "https://raw.githubusercontent.com/AI4Finance-Foundation/FinRL/refs/heads/master/requirements.txt"
!sudo apt-get install swig
!pip install box2d-py
!pip install -r requirements.txt
!pip install finrl yfinance stable_baselines3


--2025-02-06 15:00:29--  https://raw.githubusercontent.com/AI4Finance-Foundation/FinRL/refs/heads/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 729 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     729  --.-KB/s    in 0s      

2025-02-06 15:00:29 (34.9 MB/s) - ‘requirements.txt’ saved [729/729]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 62 not upgraded.
Need to get 1,116 kB of archives.
After 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import warnings

warnings.filterwarnings("ignore")
%matplotlib inline


from finrl.meta.preprocessor.yahoodownloader import YahooDownloader

ticker = "GOOGL"
benchmark_ticker = "^GSPC"  # S&P 500
start_date = "2015-01-01"
end_date = "2025-01-01"

df_stock = YahooDownloader(start_date=start_date, end_date=end_date, ticker_list=[ticker]).fetch_data()
df_benchmark = YahooDownloader(start_date=start_date, end_date=end_date, ticker_list=[benchmark_ticker]).fetch_data()


df = pd.merge(df_stock, df_benchmark[['date', 'close']], on='date', suffixes=('', '_benchmark'))

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (2516, 8)
Shape of DataFrame:  (2516, 8)


In [4]:
# Plot stock closing price over time
plt.figure(figsize=(12, 6))
sns.lineplot(x=pd.to_datetime(df['date']), y=df["close"], label="Closing Price", color="blue")
plt.xlabel("Date")
plt.ylabel("Stock Price (USD)")
plt.title(f"{ticker} Closing Stock Over Time")
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.show()

# Plot benchmark data over time
plt.figure(figsize=(12, 6))
sns.lineplot(x=pd.to_datetime(df['date']), y=df["close_benchmark"], label="Benchmark Closing Price", color="green")
plt.xlabel("Date")
plt.ylabel("Stock Price (USD)")
plt.title(f"{benchmark_ticker} Over Time")
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.show()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/seaborn/_oldcore.py:1498: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/usr/local/lib/python3.10/dist-packages/seaborn/_oldcore.py:1498: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/usr/local/lib/python3.10/dist-packages/seaborn/_oldcore.py:1498: DeprecationWarning: is_categorical_dtype is deprecated an

In [5]:
from finrl.config import INDICATORS
from finrl.meta.preprocessor.preprocessors import FeatureEngineer

fe = FeatureEngineer(
    use_technical_indicator=True, 
    tech_indicator_list=INDICATORS,
    use_turbulence=True
)
df = fe.preprocess_data(df)
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Successfully added technical indicators
Successfully added turbulence index


,date,open,high,low,close,volume,tic,day,close_benchmark,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2015-01-02,26.381865,26.477501,26.790001,26.629999,26480000,GOOGL,4,2058.899902,0.000000,26.879122,26.108378,0.0,-66.666667,100.0,26.629999,26.629999,0.0
1,2015-01-05,25.879185,25.973000,26.399500,26.357500,41182000,GOOGL,0,2054.439941,-0.006114,26.879122,26.108378,0.0,-66.666667,100.0,26.493750,26.493750,0.0
2,2015-01-06,25.240501,25.332001,26.060499,26.025000,54456000,GOOGL,1,2022.150024,-0.018101,26.943490,25.731509,0.0,-100.000000,100.0,26.337500,26.337500,0.0
3,2015-01-07,25.166269,25.257500,25.574499,25.547501,46918000,GOOGL,2,2005.550049,-0.040349,27.072156,25.207844,0.0,-95.317939,100.0,26.140000,26.140000,0.0
4,2015-01-08,25.253954,25.345501,25.375000,25.075500,73054000,GOOGL,3,2030.609985,-0.070371,27.175384,24.678816,0.0,-92.239824,100.0,25.927100,25.927100,0.0


In [6]:

from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv

class CustomStockTradingEnv(StockTradingEnv):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.reward_function = None

    def set_reward_function(self, reward_function):
        self.reward_function = reward_function

    def step(self, actions):
        
        next_state, reward, terminal, truncated, info = super().step(actions)
        if self.reward_function is not None:
            reward = self.reward_function(self, actions, next_state, reward, terminal, truncated, info)
        return next_state, reward, terminal, truncated, info

print("Loaded class CustomStockTradingEnv")

Loaded class CustomStockTradingEnv


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# Global reward logging switch and accumulator variables
reward_logging = False  # set to True to print component values during reward calculation
overall_reward = 0      # cumulative reward tracker

# Define default weights (to be tuned via grid search)
reward_weights = {
    'w1': 1.0,   # Weight for annualized returns
    'w2': 1.0,   # Weight for downside_std penalty
    'w3': 1.0,   # Weight for Treynor ratio
    'w4': 1.0    # Weight for differential return
}

def sample_custom_reward(self, actions, next_state, base_reward, terminal, truncated, info):
    import numpy as np
    import pandas as pd
   
    df_total_value = pd.DataFrame(self.asset_memory, columns=["account_value"])
    df_total_value["date"] = self.date_memory
    df_total_value["daily_return"] = df_total_value["account_value"].pct_change(1)
    
    
    df_total_value["benchmark_value"] = self.df["close_benchmark"].iloc[:len(df_total_value)].reset_index(drop=True)
    df_total_value["benchmark_daily_return"] = df_total_value["benchmark_value"].pct_change(1)
    
    remove_nan = lambda x: 0 if np.isnan(x) else x
    
    mean_returns = remove_nan(df_total_value["daily_return"].mean())
    downside_std = remove_nan(df_total_value["daily_return"][df_total_value["daily_return"] < 0].std(ddof=1))
    bench_returns = remove_nan(df_total_value["benchmark_daily_return"].mean())
    beta = 1.0
    

    if len(df_total_value) > 1:
        cov = np.cov(
            df_total_value["daily_return"].fillna(0),
            df_total_value["benchmark_daily_return"].fillna(0)
        )[0][1]
        var_bench = df_total_value["benchmark_daily_return"].var()
        beta = cov / var_bench if var_bench != 0 else 1.0
    
    # Annualize metrics
    annualized_return = (252**0.5) * mean_returns
    treynor_ratio = annualized_return / beta if beta != 0 else 0
    diff_return = (mean_returns - bench_returns) / beta if beta != 0 else 0
    
    # Apply weights
    w1, w2, w3, w4 = reward_weights.values()
    total_reward = (
        w1 * annualized_return
        - w2 * abs(downside_std)
        + w3 * treynor_ratio
        + w4 * diff_return
    )
    if reward_logging:
        print("Reward Components (weighted):")
        print(f"  Annualized Returns: {w1 * annualized_return:.6f}")
        print(f"  - Downside Std: {w2 * abs(downside_std):.6f}")
        print(f"  Treynor Ratio: {w3 * treynor_ratio:.6f}")
        print(f"  Differential Return: {w4 * diff_return:.6f}")
        print(f"  Total Reward: {total_reward:.6f}")
        print("-----------------------------")
    
    global overall_reward
    overall_reward += total_reward
    return total_reward

print("Loaded custom reward function with configurable weights")
    


Loaded custom reward function with configurable weights


In [8]:

indicators = ["volume", "macd", "boll_ub", "boll_lb", "rsi_30", "cci_30", "dx_30", 
              "close_30_sma", "close_60_sma", "turbulence"]
turbulence_threshold = 100

stock_dim = len(df["tic"].unique())
max_price = df['close'].max()
initial_amount = 10000
hmax = int(initial_amount / max_price)

def create_env():
    return CustomStockTradingEnv(
        df=df, 
        stock_dim=stock_dim, 
        hmax=hmax,
        initial_amount=initial_amount, 
        num_stock_shares=[0],
        print_verbosity=2,
        buy_cost_pct=[0.001],
        sell_cost_pct=[0.001],
        turbulence_threshold=turbulence_threshold,
        reward_scaling=1e-4,
        tech_indicator_list=indicators,
        state_space=1 + 2 * stock_dim + len(indicators) * stock_dim,
        action_space=stock_dim,
    )

env = create_env()
env.reset()
reward_logging = True
env.set_reward_function(sample_custom_reward)

_ = env.step(np.array([5]))
_ = env.step(np.array([-5]))
_ = env.step(np.array([0]))
_ = env.step(np.array([0]))
_ = env.step(np.array([0]))
reward_logging = False

Reward Components (weighted):
  Annualized Returns: -0.118713
  - Downside Std: 0.000000
  Treynor Ratio: nan
  Differential Return: nan
  Total Reward: nan
-----------------------------
Reward Components (weighted):
  Annualized Returns: -0.064626
  - Downside Std: 0.004818
  Treynor Ratio: 0.541825
  Differential Return: -0.040835
  Total Reward: 0.431546
-----------------------------
Reward Components (weighted):
  Annualized Returns: -0.043084
  - Downside Std: 0.004818
  Treynor Ratio: 0.224968
  Differential Return: -0.031243
  Total Reward: 0.145823
-----------------------------
Reward Components (weighted):
  Annualized Returns: -0.032313
  - Downside Std: 0.004818
  Treynor Ratio: -4.114871
  Differential Return: 0.173664
  Total Reward: -3.978339
-----------------------------
Reward Components (weighted):
  Annualized Returns: -0.025850
  - Downside Std: 0.004818
  Treynor Ratio: -0.788786
  Differential Return: -0.065405
  Total Reward: -0.884861
----------------------------

In [9]:

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env


vec_env = make_vec_env(lambda: create_env(), n_envs=1)

# Create the PPO agent
model = PPO("MlpPolicy", vec_env, verbose=1)


model.learn(total_timesteps=100000)

def safe_step(env, action):
    result = env.step(action)
    if len(result) == 4:

        obs, reward, done, info = result
        truncated = [False] * (len(reward) if isinstance(reward, (list, np.ndarray)) else 1)
        return obs, reward, done, truncated, info
    elif len(result) == 5:
        return result
    else:
        raise ValueError("Unexpected number of return values from env.step()")

Using cpu device


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


-----------------------------
| time/              |      |
|    fps             | 605  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.52e+03    |
|    ep_rew_mean          | 3.2         |
| time/                   |             |
|    fps                  | 486         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005827756 |
|    clip_fraction        | 0.0417      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.41       |
|    explained_variance   | 0           |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00458    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.003

In [10]:

def train_and_evaluate(vec_env, agent, train_steps=10000, eval_steps=1000):
    """
    Train the agent for 'train_steps' timesteps and then evaluate for 'eval_steps' timesteps.
    Returns the cumulative reward during evaluation.
    """
    agent.learn(total_timesteps=train_steps)
    obs = vec_env.reset()
    cumulative_reward = 0
    for _ in range(eval_steps):
        action, _ = agent.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = safe_step(vec_env, action)
 
        cumulative_reward += reward[0] if isinstance(reward, (list, np.ndarray)) else reward
        if done[0] if isinstance(done, (list, np.ndarray)) else done:
            break
    return cumulative_reward

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# Grid search over reward weights

weight_grid = {
    'w1': np.logspace(-2, 2, num=5),  # [0.01, 0.1, 1, 10, 100]
    'w2': np.logspace(-2, 2, num=5),
    'w3': np.logspace(-2, 2, num=5),
    'w4': np.logspace(-2, 2, num=5)
}

# Generate all combinations
keys, values = zip(*weight_grid.items())
weight_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

best_performance = -np.inf
best_weights = {}

print("Starting grid search over reward weights...")
for weights in weight_combinations:
    # Update global reward weights
    global reward_weights
    reward_weights = weights
    
    # Create a new vectorized environment for this grid search iteration
    gs_vec_env = make_vec_env(lambda: create_env(), n_envs=1)
    # Create a new PPO agent
    agent = PPO("MlpPolicy", gs_vec_env, verbose=0)
    
    # Train and evaluate the agent with the current weight combination
    performance = train_and_evaluate(gs_vec_env, agent, train_steps=10000, eval_steps=1000)
    
    print(f"Tested weights: {weights} | Performance: {performance:.4f}")
    
    # Update best weights if performance improves
    if performance > best_performance:
        best_performance = performance
        best_weights = weights

print(f"\nBest weights: {best_weights} with performance {best_performance:.4f}")


Starting grid search over reward weights...
day: 2515, episode: 2
begin_total_asset: 10000.00
end_total_asset: 9840.49
total_reward: -159.51
total_cost: 4545.27
total_trades: 2202
Sharpe: 0.065
day: 2515, episode: 4
begin_total_asset: 10000.00
end_total_asset: 32771.03
total_reward: 22771.03
total_cost: 5500.10
total_trades: 2414
Sharpe: 0.728
Tested weights: {'w1': 0.01, 'w2': 0.01, 'w3': 0.01, 'w4': 0.01} | Performance: 0.9259
day: 2515, episode: 2
begin_total_asset: 10000.00
end_total_asset: 25005.33
total_reward: 15005.33
total_cost: 5170.23
total_trades: 2444
Sharpe: 0.535
day: 2515, episode: 4
begin_total_asset: 10000.00
end_total_asset: 57133.23
total_reward: 47133.23
total_cost: 4291.40
total_trades: 2474
Sharpe: 0.810
Tested weights: {'w1': 0.01, 'w2': 0.01, 'w3': 0.01, 'w4': 0.1} | Performance: 0.9879
day: 2515, episode: 2
begin_total_asset: 10000.00
end_total_asset: 14005.03
total_reward: 4005.03
total_cost: 4787.72
total_trades: 2392
Sharpe: 0.270
day: 2515, episode: 4
begi

In [12]:

obs = vec_env.reset()
vec_env.envs[0].unwrapped.episode = 0  
overall_reward = 0  

portfolio_values = []
timesteps = []

i = 0
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, done, truncated, info = safe_step(vec_env, action)
    if done[0] if isinstance(done, (list, np.ndarray)) else done:
        break
   
    portfolio_value = vec_env.envs[0].unwrapped.asset_memory[-1]
    portfolio_values.append(portfolio_value)
    timesteps.append(i)
    i += 1

plt.figure(figsize=(10, 5))
plt.plot(timesteps, portfolio_values, label="Portfolio Value", color='blue')
plt.xlabel("Time Steps")
plt.ylabel("Portfolio Value ($)")
plt.title("RL Model Performance")
plt.legend()
plt.grid()
plt.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


day: 2515, episode: 0
begin_total_asset: 10000.00
end_total_asset: 72428.72
total_reward: 62428.72
total_cost: 9.98
total_trades: 2515
Sharpe: 0.838
